In [ ]:
%env CUDA_VISIBLE_DEVICES=0

import numpy as np
import networkx as nx
import pandas as pd
import cudf
import cugraph
import time
from cuxfilter.charts.datashader.custom_extensions.graph_assets import calc_connected_edges

# import plotly.express as px
# import plotly.graph_objects as go

ITERATIONS=500
THETA=1.0
OPTIMIZE=True

import holoviews as hv

from colorcet import fire

from holoviews.operation.datashader import datashade, dynspread
from holoviews.operation import decimate

from dask.distributed import Client
import networkx as nx


client = Client()
hv.notebook_extension('bokeh', 'matplotlib')
decimate.max_samples=20000
dynspread.threshold=0.01
datashade.cmap=fire[40:]
sz = dict(width=150,height=150)
%opts RGB [xaxis=None yaxis=None show_grid=False bgcolor="black"]


In [ ]:
!head music_graph_min_number_of_connections_10_giant_connected_component.csv

In [ ]:
music_graph_edges = pd.read_csv("./music_graph_min_number_of_connections_10_giant_connected_component.csv", index_col=None, header=None)
music_graph_edges.columns=["source", "target"]
music_graph_edges.head()

In [ ]:
G = cugraph.Graph()
G.from_pandas_edgelist(music_graph_edges, source="source", destination="target", renumber=True)

In [ ]:
G.degree().to_pandas()

In [ ]:
G.degree().to_pandas()["degree"].value_counts().to_dict()

In [ ]:
G.degree().to_pandas().groupby("degree").count().to_dict()["vertex"]

In [ ]:
import typing as tp

In [ ]:
def get_degree_distribution(G: cugraph.Graph) -> tp.Sequence[int | float]:
    degree_to_count: dict[int, int] = G.degree().to_pandas()["degree"].value_counts().to_dict()
    max_number_of_connections = G.number_of_nodes() - 1
    degree_vector: tp.Sequence[int | float] = np.zeros(max_number_of_connections)

    for degree, count in degree_to_count.items():
        degree_vector[degree] = count
    return degree_vector

In [ ]:
degree_vector = get_degree_distribution(G)

In [ ]:
import plotly.express as px

In [ ]:
px.scatter(x=np.arange(G.number_of_nodes()-1), y=degree_vector, log_x=True, log_y=True, title="Degree distribution of Music graph (unnormalized)")

In [ ]:
px.scatter(x=np.arange(G.number_of_nodes()-1), y=degree_vector / G.number_of_nodes(), log_x=True, log_y=True, title="Degree distribution of Music graph (normalized)")

In [ ]:
G.graph_properties

In [ ]:
G.nodes().values

In [ ]:
a, b, c = cugraph.node2vec(G, start_vertices=G.nodes(), max_depth=5)

In [ ]:
a

In [ ]:
b

In [ ]:
c